In [4]:
!pip install ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.0 MB/s eta 0:00:00


In [5]:
%load_ext autotime

time: 486 µs (started: 2023-12-01 15:24:39 +00:00)


In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas.core.frame import DataFrame
import nltk
nltk.download('punkt')
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pandas.core.frame import DataFrame
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk import ngrams


from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

time: 239 ms (started: 2023-12-01 16:47:35 +00:00)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
#  stemming, tokenizing and make bigrams for each plot

def stemm_text(text):
    ps = PorterStemmer()
    tokenizer = RegexpTokenizer(r"(?u)\b\w\w+\b")
    stemmized_list= [ps.stem(w) for w in tokenizer.tokenize(text)]
    n_grams = ngrams(stemmized_list, 2)
    concat_list=stemmized_list+ [ ' '.join(grams) for grams in n_grams]
    return concat_list

# stems incoming words, splits two word inputs as well
def similarity_calculator(token1,token2,vocab):
    ps = PorterStemmer()
    token1=" ".join([ps.stem(w) for w in token1.split()])
    token2=" ".join([ps.stem(w) for w in token2.split()])

    cosine_to_return=None
    if token1.isnumeric():
      token1='numberspecialtoken'
    if token2.isnumeric():
      token2='numberspecialtoken'
    try:
      cosine_to_return=cosine_similarity([vocab[token1]],[vocab[token2]])
    except KeyError:
      cosine_to_return=np.array([[0.0]])
    return cosine_to_return[0,0]




time: 2.79 ms (started: 2023-12-01 16:47:37 +00:00)


In [39]:
TRAINING_DATA_FILE_PATH='./data/Training-dataset.csv'

VALIDATION_DATA_FILE_PATH='./data/Task-1-validation-dataset.csv'
VALIDATION_DATA_FILE_PATH_OUTPUT='./data/10664226-Task1-method-a-validation.csv'

TEST_DATA_FILE_PATH='./data/Task-1-test-dataset2.csv'
TEST_DATA_FILE_PATH_OUTPUT='./data/10664226-Task1-method-a.csv'

training_data=pd.read_csv(TRAINING_DATA_FILE_PATH)
validation_data=pd.read_csv(VALIDATION_DATA_FILE_PATH)
testing_data=pd.read_csv(TEST_DATA_FILE_PATH)



time: 592 ms (started: 2023-12-01 16:47:39 +00:00)


In [40]:
training_data['plot_synopsis']=training_data['title']+training_data['plot_synopsis']
training_data_for_tfidf=training_data

# preprocessing
training_data_for_tfidf['plot_synopsis'].replace('[0-9][0-9,]*',' NUMBERSPECIALTOKEN ',regex=True, inplace = True)
training_data_for_tfidf['plot_synopsis'].replace('[_&+=][_&+=]*','.',regex=True, inplace= True)

#processing
tfidf_vectorizer= TfidfVectorizer (tokenizer=stemm_text, lowercase=True)


time: 1.53 s (started: 2023-12-01 16:47:41 +00:00)


In [41]:
tfidf_vocabulary= tfidf_vectorizer.fit_transform(training_data_for_tfidf['plot_synopsis'])


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


time: 3min 17s (started: 2023-12-01 16:47:43 +00:00)


In [42]:
#operations to align the new (document,tokens) structure with names of the columns (tokens)
all_tokens=tfidf_vectorizer.get_feature_names_out()


time: 3.9 s (started: 2023-12-01 16:51:00 +00:00)


In [43]:
df_vocabulary=pd.DataFrame.sparse.from_spmatrix(tfidf_vocabulary, columns=all_tokens)


time: 25.7 s (started: 2023-12-01 16:51:04 +00:00)


In [44]:
def predict_and_write(data_to_predict,fabricated_columns_for_data_to_predict, FILE_PATH_OUTPUT):

  columns_array=data_to_predict.columns.values

  data_to_predict_df=pd.DataFrame(data=data_to_predict.to_numpy(), columns=fabricated_columns_for_data_to_predict)

  result_list=[]
  # iterating through the data rows and calculating similarity row wise
  for ind in data_to_predict.index:
    res=similarity_calculator(data_to_predict_df['word1'][ind],data_to_predict_df['word2'][ind], df_vocabulary)
    result_list.append([data_to_predict_df['id'][ind], res*10])


  df_out=pd.DataFrame(data=result_list, columns=[columns_array[0],similarity_calculator(columns_array[1],columns_array[2], df_vocabulary)])
  df_out.to_csv(FILE_PATH_OUTPUT,index=False)


time: 761 µs (started: 2023-12-01 16:51:29 +00:00)


In [45]:
predict_and_write(validation_data,['id','word1','word2','score'], VALIDATION_DATA_FILE_PATH_OUTPUT)



time: 3.19 s (started: 2023-12-01 16:51:29 +00:00)


In [46]:
predict_and_write(testing_data,['id','word1','word2'], TEST_DATA_FILE_PATH_OUTPUT)

time: 84.1 ms (started: 2023-12-01 16:51:33 +00:00)


In [47]:
tfidf_vocabulary.size

8536433

time: 3.16 ms (started: 2023-12-01 16:51:33 +00:00)


Start of Task1 Method b Word 2 Vec Continuous bag of words

In [30]:
TRAINING_DATA_FILE_PATH='./data/Training-dataset.csv'

VALIDATION_DATA_FILE_PATH='./data/Task-1-validation-dataset.csv'
VALIDATION_DATA_FILE_PATH_OUTPUT='./data/10664226-Task1-method-b-validation.csv'

TEST_DATA_FILE_PATH='./data/Task-1-test-dataset2.csv'
TEST_DATA_FILE_PATH_OUTPUT='./data/10664226-Task1-method-b.csv'

training_data=pd.read_csv(TRAINING_DATA_FILE_PATH)
validation_data=pd.read_csv(VALIDATION_DATA_FILE_PATH)
testing_data=pd.read_csv(TEST_DATA_FILE_PATH)

training_data['plot_synopsis']=training_data['title']+" "+training_data['plot_synopsis']



time: 830 ms (started: 2023-12-01 16:42:51 +00:00)


In [31]:
# first tokenize sentences then tokenize each sentence
sent_tokenized_trained_data=training_data['plot_synopsis'].apply(sent_tokenize)
list_of_tokenized_sentences=sent_tokenized_trained_data.explode("plot_sypnosis").tolist()
list_of_tokenized_sentences_and_words=[]

for sentence in list_of_tokenized_sentences:
  tokenized_list= word_tokenize(sentence)
  tokenized_list =[token.lower() for token in tokenized_list]
  list_of_tokenized_sentences_and_words.append(tokenized_list)

time: 1min 18s (started: 2023-12-01 16:42:53 +00:00)


In [32]:
# context window is 4 each side, word embedding (vector_size) based on testing of accuracy
model = Word2Vec(
    sentences = list_of_tokenized_sentences_and_words,
    vector_size = 100,
    window = 4,
)

time: 1min 5s (started: 2023-12-01 16:44:11 +00:00)


In [33]:
# for 2 word inputs, each word's embedding gets extracted and then the embeddings are concatenated and compared with the other input

def cosine_similarity_word2vec(word1,word2,model):

  concat_array_word1=np.array([])
  concat_array_word2=np.array([])
  word1_list=word1.split()
  word2_list=word2.split()
  word1_flag=False
  word2_flag=False
  res=None
  try:
    for idx,word in enumerate(word1.split()):
      word=word.lower()
      concat_array_word1 =np.concatenate((concat_array_word1, model.wv[word]))
      if idx>0:
        word1_flag=True

    for idx,word in enumerate(word2.split()):
      word=word.lower()
      concat_array_word2 =np.concatenate((concat_array_word2, model.wv[word]))
      if idx>0:
        word2_flag=True

    res=None
    # shape needs to be equal so padding needs to be applied on the smaller array
    if word1_flag==True and word2_flag==False:
      shape_diff=concat_array_word1.shape[0]-concat_array_word2.shape[0]
      concat_array_word2=np.pad(concat_array_word2, (0, shape_diff), 'constant',  constant_values=(0, 0))

    elif word1_flag==False and word2_flag==True:
      shape_diff=concat_array_word2.shape[0]-concat_array_word1.shape[0]
      concat_array_word1=np.pad(concat_array_word1, (0, shape_diff), 'constant',  constant_values=(0, 0))



    res=cosine_similarity([concat_array_word1],[concat_array_word2])



  except KeyError:
    res=[[0.0]]



  return res[0][0]


time: 2.13 ms (started: 2023-12-01 16:45:16 +00:00)


In [34]:
def predict_and_write_word2Vec(data_to_predict,fabricated_columns_for_data_to_predict, FILE_PATH_OUTPUT,model):

  columns_array=data_to_predict.columns.values

  data_to_predict_df=pd.DataFrame(data=data_to_predict.to_numpy(), columns=fabricated_columns_for_data_to_predict)

  result_list=[]

  for ind in data_to_predict.index:
    res=cosine_similarity_word2vec(data_to_predict_df['word1'][ind],data_to_predict_df['word2'][ind], model)
    result_list.append([data_to_predict_df['id'][ind], res*10])


  df_out=pd.DataFrame(data=result_list, columns=[columns_array[0],cosine_similarity_word2vec(columns_array[1],columns_array[2], model)])
  df_out.to_csv(FILE_PATH_OUTPUT,index=False)



time: 1.31 ms (started: 2023-12-01 16:45:16 +00:00)


In [35]:
predict_and_write_word2Vec(validation_data,['id','word1','word2','score'], VALIDATION_DATA_FILE_PATH_OUTPUT,model)


time: 102 ms (started: 2023-12-01 16:45:16 +00:00)


In [36]:
predict_and_write_word2Vec(testing_data,['id','word1','word2'], TEST_DATA_FILE_PATH_OUTPUT,model)



time: 66 ms (started: 2023-12-01 16:45:16 +00:00)
